## Load the CSV data

In [5]:
import pandas as pd

In [2]:
ZOO_DATA = '/home/torradeflot/Downloads/ZooSpecPhotoDR19_torradeflot.csv'

In [3]:
df = pd.read_csv(ZOO_DATA)

# Filter out incorrect data

According to the documentation, we will be using the Model magnitude. We filter out the objects without a proper magnitude: https://www.sdss4.org/dr12/algorithms/magnitudes/

We will apply a restrictive threshold in the zooSpec table to decide if an object was successfully classified

We will only keep the objects that are reasonable big, but still fit comfortably in the cutouts.

In [4]:
m = (
    (df.modelMag_u >-30) & (df.modelMag_g > -30) & (df.modelMag_r > -30) & (df.modelMag_i > -30) & (df.modelMag_z > -30) # correct magnitudes
    & (df.modelMagErr_u < 0.5) & (df.modelMagErr_g < 0.05) & (df.modelMagErr_r < 0.05) & (df.modelMagErr_i < 0.05) & (df.modelMagErr_z < 0.1) # reasonable errors
    & ((df.p_cs_debiased >= 0.9) | (df.p_el_debiased >= 0.9)) # very certain about the classification
    & (df.petroR90_r*2*1.5/0.4 < 64) & (df.petroR90_r*2/0.4 > 20) # Medium sized
)

In [5]:
len(df), len(df[m])

(659272, 69352)

In [6]:
cols_to_keep = ['specobjid', 'objid', 'dr7objid', 'ra', 'dec', 'p_el_debiased', 'p_cs_debiased', 'spiral', 'elliptical'] + \
    ['petroR50_r', 'petroR90_r'] + [f'modelMag_{f}' for f in "ugriz"] + [f'extinction_{f}' for f in "ugriz"]
df_filtered = df[m][cols_to_keep]

In [7]:
df_filtered.to_csv('resources/ZooSpecPhotoDR19_filtered.csv')

In [ ]:
import os
from PIL import Image

image_path = "C:\Users\marce\Desktop\HEP\Sem1 - Statistics\coding_project\final_code\HIPS2FITS_petro\00\0.jpg"
img = Image.open(image_path).convert('L')

img_array = np.array(img)

U,sigma,Vt = np.linalg.svd(img)

k=10

Uk = U[:,:k]
sigmak = np.diag(sigma[:k])
Vtk = Vt[:k,:]

A_dn = Uk @ sigmak @ Vtk



In [45]:
import pandas
import numpy as np
import os
from PIL import Image

In [47]:
w = pd.read_csv(r"C:\Users\marce\Desktop\HEP\Sem1 - Statistics\coding_project\final_code\ZooSpecPhotoDR19_filtered.csv",dtype={'dr7objid':str})

In [61]:
def svd_denoise_features(df, id_col, data_dir, k, grayscale = True):
    if not (0 < k <= 64):
        print(f"{k} needs to be between 0 and 64")

    svd_feature_list = []

    for index, row in df.iterrows():
        print(f'Starting SVD for row {index+1}/{len(df)}\r')
        raw_id = row[id_col]

        img_id_str = str(raw_id)
        subfolder = img_id_str[-2:]
        
        img_name = str(raw_id)+".jpg"
        subfolder_path = os.path.join(data_dir, subfolder)
        img_path = os.path.join(subfolder_path, img_name)

        # now load images
        img = Image.open(img_path)
        if grayscale: # reduce dimensions by converting to grayscale
            A = np.array(img.convert('L'), dtype=np.float64)
        
        # SVD
        sigma = np.linalg.svd(A, compute_uv=False) # U and V are the upper and lower triangular matrices associated with the images, not necessary
        top_k_vals = sigma[:k]
        svd_row = {id_col: raw_id}
        for i in range(k):
            svd_row[f'svd_comp_{i+1}'] = top_k_vals[i]
        svd_feature_list.append(svd_row)

    print("SVD Complete")
    svd_df = pd.DataFrame(svd_feature_list)
    svd_df.to_csv(f'SVD_features_{k}.csv')
    print("SVD features saved as csv")
    return svd_df


In [63]:
data_dir = r"C:\Users\marce\Desktop\HEP\Sem1 - Statistics\coding_project\final_code\HIPS2FITS_petro"
k = 20
id_col = 'objid'
svd_results = svd_denoise_features(w,id_col=id_col,data_dir=data_dir,k=k)

Starting SVD for row 1/69352
Starting SVD for row 2/69352
Starting SVD for row 3/69352
Starting SVD for row 4/69352
Starting SVD for row 5/69352
Starting SVD for row 6/69352
Starting SVD for row 7/69352
Starting SVD for row 8/69352
Starting SVD for row 9/69352
Starting SVD for row 10/69352
Starting SVD for row 11/69352
Starting SVD for row 12/69352
Starting SVD for row 13/69352
Starting SVD for row 14/69352
Starting SVD for row 15/69352
Starting SVD for row 16/69352
Starting SVD for row 17/69352
Starting SVD for row 18/69352
Starting SVD for row 19/69352
Starting SVD for row 20/69352
Starting SVD for row 21/69352
Starting SVD for row 22/69352
Starting SVD for row 23/69352
Starting SVD for row 24/69352
Starting SVD for row 25/69352
Starting SVD for row 26/69352
Starting SVD for row 27/69352
Starting SVD for row 28/69352
Starting SVD for row 29/69352
Starting SVD for row 30/69352
Starting SVD for row 31/69352
Starting SVD for row 32/69352
Starting SVD for row 33/69352
Starting SVD for ro

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\marce\\Desktop\\HEP\\Sem1 - Statistics\\coding_project\\final_code\\HIPS2FITS_petro\\0\\0.jpg'

In [45]:
import pandas as pd
import numpy as np
import os
from PIL import Image

w = pd.read_excel(r"C:\Users\marce\Desktop\HEP\Sem1 - Statistics\coding_project\final_code\ZooSpecPhotoDR19_filtered.xlsx",dtype={'objid':str})


from sklearn.model_selection import train_test_split



w = w.drop(columns=['spiral','elliptical'])



def svd_denoise_features(df, id_col, data_dir, k, grayscale = True):
    if not (0 < k <= 64):
        print(f"{k} needs to be between 0 and 64")

    svd_feature_list = []
    missing_files = []

    for index, row in df.iterrows():
        print(f'Starting SVD for row {index+1}/{len(df)}',end='\r')
        raw_id = row[id_col]

        img_id_str = str(int(raw_id)).zfill(2)
        subfolder = img_id_str[-2:]
        
        img_name = str(img_id_str)+".jpg"
        subfolder_path = os.path.join(data_dir, subfolder)
        img_path = os.path.join(subfolder_path, img_name)

        # if no image associated with objid, save the id and skip
        if not os.path.exists(img_path):
            missing_files.append(img_path)
            continue

        try:
            # now load images
            img = Image.open(img_path)
            if grayscale: # reduce dimensions by converting to grayscale
                A = np.array(img.convert('L'), dtype=np.float64)
            
            # SVD
            sigma = np.linalg.svd(A, compute_uv=False) # U and V are the upper and lower triangular matrices associated with the images, not necessary
            top_k_vals = sigma[:k]
            svd_row = {id_col: raw_id}
            for i in range(k):
                svd_row[f'svd_comp_{i+1}'] = top_k_vals[i]
            svd_feature_list.append(svd_row)
        except Exception as e:
            print(f"\nError processing {img_path}: {e}")
            missing_files.append(img_path)
            continue

    print("SVD Complete")
    svd_df = pd.DataFrame(svd_feature_list)
    svd_df.to_csv(f'SVD_features_{k}.csv')
    print("SVD features saved as csv")
    return svd_df


data_dir = r"C:\Users\marce\Desktop\HEP\Sem1 - Statistics\coding_project\final_code\HIPS2FITS_petro"
k = 20
id_col = 'objid'
svd_results = svd_denoise_features(w,id_col=id_col,data_dir=data_dir,k=k)

SVD Complete for row 69352/69352
SVD features saved as csv


In [1]:
import numpy as np
import pandas as pd

In [2]:
label_map = {'uncertain':0,
             'spiral':1,
             'elliptical':2}

In [5]:
svd_results = pd.read_csv(r"C:\Users\marce\Desktop\HEP\Sem1 - Statistics\coding_project\final_code\SVD_features_20.csv")
w = pd.read_excel(r"C:\Users\marce\Desktop\HEP\Sem1 - Statistics\coding_project\final_code\ZooSpecPhotoDR19_filtered.xlsx")

In [7]:
w_with_svd = w.merge(svd_results,how='inner',on='objid')
w_with_svd['target'] = w_with_svd['target'].map(label_map)
w_with_svd['target'] = w_with_svd['target'].astype(int)

In [9]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
train_split = 0.6
val_split = 0.2
test_split = 0.2

X = w_with_svd.copy()
X = X.drop(columns=['objid','dr7objid','specobjid','ra','dec','spiral','elliptical'])

train, temp = train_test_split(X, test_size=(1-train_split), stratify=X['target'],random_state=111)

rel_val_size = val_split / (val_split + test_split)
val, test = train_test_split(temp, test_size=(1-rel_val_size), stratify=temp['target'],random_state =111)

train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

In [17]:
X_train = train.drop(columns=['target'])
y_train = train['target']

X_val = val.drop(columns=['target'])
y_val = val['target']

X_test = test.drop(columns=['target'])
y_test = test['target']

y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)


In [19]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [21]:

rf = RandomForestClassifier(random_state=111)
svm = SVC(kernel='rbf', random_state=111)
logreg = LogisticRegression(max_iter=500, multi_class='multinomial', random_state=111)

# Hyperparameter grids
param_grids = {
    'Random Forest': {'n_estimators': [100, 200], 'max_depth': [None, 10, 20]},
    'SVM': {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']},
    'Logistic Regression': {'C': [0.1, 1, 10]}
}

In [23]:
from sklearn.metrics import accuracy_score

best_models = {}

# Random Forest
best_acc = 0
for n in [100, 150, 200, 250]:
    for d in [1, 10, 15]:
        rf_tmp = RandomForestClassifier(n_estimators=n, max_depth=d, random_state=111)
        rf_tmp.fit(X_train, y_train)
        y_val_pred = rf_tmp.predict(X_val)
        acc = accuracy_score(y_val, y_val_pred)
        if acc > best_acc:
            best_acc = acc
            best_models['Random Forest'] = rf_tmp
print(f"Best RF val acc: {best_acc:.3f}")

# SVM
best_acc = 0
for C in [0.1, 1, 3, 5, 7, 10]:
    for gamma in ['scale', 'auto']:
        svm_tmp = SVC(C=C, gamma=gamma, kernel='rbf', random_state=111)
        svm_tmp.fit(X_train_scaled, y_train)
        y_val_pred = svm_tmp.predict(X_val_scaled)
        acc = accuracy_score(y_val, y_val_pred)
        if acc > best_acc:
            best_acc = acc
            best_models['SVM'] = svm_tmp
print(f"Best SVM val acc: {best_acc:.3f}")

# Logistic Regression
best_acc = 0
for C in [0.1, 1, 3, 5, 7, 10]:
    logreg_tmp = LogisticRegression(C=C, max_iter=500, multi_class='multinomial', random_state=111)
    logreg_tmp.fit(X_train_scaled, y_train)
    y_val_pred = logreg_tmp.predict(X_val_scaled)
    acc = accuracy_score(y_val, y_val_pred)
    if acc > best_acc:
        best_acc = acc
        best_models['Logistic Regression'] = logreg_tmp
print(f"Best Logistic Regression val acc: {best_acc:.3f}")


Best RF val acc: 0.963
Best SVM val acc: 0.960


C:\Users\marce\anaconda3\envs\pythonbootcamp\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\marce\anaconda3\envs\pythonbootcamp\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\marce\anaconda3\envs\pythonbootcamp\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\marce\anaconda3\envs\pythonbootcamp\Lib\site-packages\sklearn\linea

Best Logistic Regression val acc: 0.962


In [24]:
from sklearn.metrics import classification_report

for name, model in best_models.items():
    if name in ['SVM', 'Logistic Regression']:
        X_used = X_test_scaled
    else:
        X_used = X_test

    y_pred = model.predict(X_used)
    print(f"\n=== {name} Test Results ===")
    print(classification_report(y_test, y_pred))



=== Random Forest Test Results ===
              precision    recall  f1-score   support

           0       0.73      0.24      0.37       619
           1       0.98      1.00      0.99     10812
           2       0.90      0.98      0.94      2562

    accuracy                           0.96     13993
   macro avg       0.87      0.74      0.77     13993
weighted avg       0.96      0.96      0.95     13993


=== SVM Test Results ===
              precision    recall  f1-score   support

           0       0.73      0.19      0.31       619
           1       0.98      1.00      0.99     10812
           2       0.90      0.98      0.94      2562

    accuracy                           0.96     13993
   macro avg       0.87      0.73      0.74     13993
weighted avg       0.95      0.96      0.95     13993


=== Logistic Regression Test Results ===
              precision    recall  f1-score   support

           0       0.64      0.36      0.46       619
           1       0.98  

In [25]:
# 1. Check for duplicates between train and test


# 2. Correlation of features with target
corrs = X.corrwith(X['target'])
print(corrs.sort_values(ascending=False).head(10))


target           1.000000
p_el_debiased    0.681637
svd_comp_1       0.203409
modelMag_u       0.166361
svd_comp_20      0.099418
svd_comp_19      0.096935
svd_comp_18      0.095228
svd_comp_17      0.093834
svd_comp_16      0.091567
svd_comp_15      0.087852
dtype: float64


In [26]:
train.columns

Index(['Unnamed: 0_x', 'p_el_debiased', 'p_cs_debiased', 'petroR50_r',
       'petroR90_r', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
       'modelMag_z', 'extinction_u', 'extinction_g', 'extinction_r',
       'extinction_i', 'extinction_z', 'target', 'Unnamed: 0_y', 'svd_comp_1',
       'svd_comp_2', 'svd_comp_3', 'svd_comp_4', 'svd_comp_5', 'svd_comp_6',
       'svd_comp_7', 'svd_comp_8', 'svd_comp_9', 'svd_comp_10', 'svd_comp_11',
       'svd_comp_12', 'svd_comp_13', 'svd_comp_14', 'svd_comp_15',
       'svd_comp_16', 'svd_comp_17', 'svd_comp_18', 'svd_comp_19',
       'svd_comp_20'],
      dtype='object')

In [31]:
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score

print(classification_report(y_test, y_pred))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.64      0.36      0.46       619
           1       0.98      1.00      0.99     10812
           2       0.92      0.97      0.94      2562

    accuracy                           0.96     13993
   macro avg       0.85      0.77      0.80     13993
weighted avg       0.96      0.96      0.96     13993

Balanced Accuracy: 0.7746019570028335
[[  224   175   220]
 [   36 10776     0]
 [   89     0  2473]]
